# Init

Start

In [2]:
import sys
import pprint
from pathlib import Path

sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

import ipytest
ipytest.autoconfig()

from context import Context
from utils.utils_chain import WrapperAddress as Address, Account, base64_to_hex
from utils.logger import get_logger

from contracts.staking_contract import StakingContract

from multiversx_sdk import ProxyNetworkProvider, ApiNetworkProvider

logger = get_logger("manual_interactor")

SIMULATOR_URL = "http://localhost:8085"
SIMULATOR_API = "http://localhost:3001"
GENERATE_BLOCKS_URL = f"{SIMULATOR_URL}/simulator/generate-blocks"
PROJECT_ROOT = Path.cwd().parent.parent
proxy = ProxyNetworkProvider(SIMULATOR_URL)
api = ApiNetworkProvider(SIMULATOR_API)

context = Context()
context.network_provider.proxy = proxy
context.network_provider.api = api

2025-03-31 10:03:05,403 - WARNING - Structure definition for locked_assets not found in /home/ubuntu/Workspace/mx-sdk-py-exchange/deploy/configs-mainnet-test/deploy_structure.json
2025-03-31 10:03:05,404 - WARNING - Structure definition for proxies not found in /home/ubuntu/Workspace/mx-sdk-py-exchange/deploy/configs-mainnet-test/deploy_structure.json
2025-03-31 10:03:05,405 - WARNING - Structure definition for proxies_v2 not found in /home/ubuntu/Workspace/mx-sdk-py-exchange/deploy/configs-mainnet-test/deploy_structure.json
2025-03-31 10:03:05,405 - WARNING - Structure definition for router not found in /home/ubuntu/Workspace/mx-sdk-py-exchange/deploy/configs-mainnet-test/deploy_structure.json
2025-03-31 10:03:05,405 - WARNING - Structure definition for pairs not found in /home/ubuntu/Workspace/mx-sdk-py-exchange/deploy/configs-mainnet-test/deploy_structure.json
2025-03-31 10:03:05,406 - WARNING - Structure definition for pairs_view not found in /home/ubuntu/Workspace/mx-sdk-py-exchan

Shadowfork detected. Owner: erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97
loaded 1 accounts from 1 PEM files.


2025-03-31 10:03:06,363 - DEBUG - Account.sync_nonce() done: 4735
2025-03-31 10:03:06,602 - DEBUG - Account.sync_nonce() done: 4735
2025-03-31 10:03:06,603 - INFO - Loaded 11 tokens.
2025-03-31 10:03:06,874 - DEBUG - Account.sync_nonce() done: 4735
2025-03-31 10:03:06,875 - INFO - Loaded 3 egld_wraps out of expected 3.
2025-03-31 10:03:06,876 - INFO - Loaded 0 locked_assets out of expected 0.
2025-03-31 10:03:06,876 - INFO - Loaded 0 proxies out of expected 0.
2025-03-31 10:03:06,877 - INFO - Loaded 0 simple_locks out of expected 0.
2025-03-31 10:03:06,877 - INFO - Loaded 1 simple_locks_energy out of expected 1.
2025-03-31 10:03:06,878 - INFO - Loaded 1 fees_collectors out of expected 1.
2025-03-31 10:03:06,878 - INFO - Loaded 1 unstakers out of expected 1.
2025-03-31 10:03:06,879 - INFO - Loaded 0 proxies_v2 out of expected 0.
2025-03-31 10:03:06,879 - INFO - Loaded 1 position_creator out of expected 1.
2025-03-31 10:03:06,880 - INFO - Loaded 0 locked_token_position_creator out of exp

In [3]:
farm_contract: StakingContract = context.deploy_structure.get_deployed_contract_by_index(config.STAKINGS_V2, 0)

pprint.pprint(farm_contract.get_config_dict())

{'address': 'erd1qqqqqqqqqqqqqpgqcedkmj8ezme6mtautj79ngv7fez978le2jps8jtawn',
 'farm_token': 'SUTK-ba35f3',
 'farming_token': 'UTK-2f80e9',
 'max_apr': 2500,
 'rewards_per_block': 13318000000000000000,
 'unbond_epochs': 10,
 'version': 2}


Permission Hub

In [5]:
from contracts.permissions_hub_contract import PermissionsHubContract

def deploy_permissions_hub():
    permissions_hub_contract = PermissionsHubContract("")
    _, address = permissions_hub_contract.contract_deploy(context.deployer_account, context.network_provider.proxy, 
                                            "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.2-rc2/permissions-hub.wasm",
                                            [])
    permissions_hub_contract.address = address
    return permissions_hub_contract

In [6]:
def set_permissions_hub(permissions_hub_contract: PermissionsHubContract):
    farm_contract.set_permissions_hub_address(context.deployer_account, context.network_provider.proxy, permissions_hub_contract.address)

Collect users

In [ ]:
from typing import List
from utils.utils_scenarios import FetchedUser, collect_farm_contract_users
from multiversx_sdk import ApiNetworkProvider

context = Context()
mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
fetched_users = collect_farm_contract_users(200, farm_contract.address, farm_contract.farmed_token, farm_contract.farm_token,
                                            mainnet_api, context.network_provider.proxy)

users_farm_tokens: List[FetchedUser] = fetched_users.get_users_with_farm_tokens()
users_farmed_tokens: List[FetchedUser] = fetched_users.get_users_with_farming_tokens()
users_both_tokens: List[FetchedUser] = fetched_users.get_users_with_both_tokens()

for user in users_farm_tokens:
    print(user.address)
print("////////////")
for user in users_farmed_tokens:
    print(user.address)
print("////////////")
for user in users_both_tokens:
    print(user.address)

# Reset context
context.network_provider.proxy = proxy
context.network_provider.api = api

Chain Config Setup

In [ ]:
import json
from typing import Any
from utils.utils_chain import WrapperAddress

USERS = ["erd1czs7ppdy4dsfwfucttd4ajt3n3hv3d693gttvzs4a0z7k9pnwzascpnfpw", # user with farm position 
         "erd1f4mm5474tt9am7wrvvm0xyvyvuy5436mh3xuzdp4eenm7qnstkesxahur5", # user with farmed tokens
         "erd1ed3ej89qyd0s48qy0ykupthpg4lfme6c6f9xhdsljyadh0m06xss70y6y7" # user with both tokens
        ]
def load_accounts_state(project_root: Path, addresses: list[str]) -> list[dict[str, Any]]:
    states = []
    
    for address in addresses:
        print(f"Loading state for {address}")
        user_path = f"0_{address}_0_chain_config_state.json"
        system_account_path = f"0_system_account_state_{address}.json"
        
        user_file = project_root / "states" / user_path
        system_file = project_root / "states" / system_account_path
        
        if user_file.exists():
            with open(user_file, "r") as file:
                user_state = json.load(file)
                if user_state:
                    print(f"Found {user_file.name}")
                    states.append(user_state)
                
        if system_file.exists():
            with open(system_file, "r") as file:
                system_state = json.load(file)
                if system_state:
                    print(f"Found {system_file.name}")
                    states.append(system_state)
            
    return states
    

def apply_states(proxy: ProxyNetworkProvider, states: list[dict[str, Any]]):
    for state in states:
        proxy.do_post(f"{SIMULATOR_URL}/simulator/set-state", state)


def load_and_apply_state(proxy: ProxyNetworkProvider, project_root: Path, owner: str, users: list[str]):
    # Load and set state for all keys
    with open(project_root / "states" / "0_all_all_keys.json", "r") as file:
        retrieved_state = json.load(file)
        apply_states(proxy, [retrieved_state])

    # Load owner and users state
    accounts = [owner]
    accounts.extend(users)
    states = load_accounts_state(project_root, accounts)
    apply_states(proxy, states)
        

def setup_chain_sim():
    # generate blocks to pass an epoch and the smart contract deploys to be enabled
    proxy.do_post(f"{GENERATE_BLOCKS_URL}/5", {})

    load_and_apply_state(proxy, PROJECT_ROOT,
                         context.deployer_account.address.bech32(),
                         USERS)


def advance_blocks(number_of_blocks: int):
    proxy.do_post(f"{GENERATE_BLOCKS_URL}/{number_of_blocks}", {})

def advance_epoch(number_of_epochs: int):
    proxy.do_post(f"{GENERATE_BLOCKS_URL}/{number_of_epochs * 20}", {})


def users_init() -> list[Account]:
    context.deployer_account.sync_nonce(context.network_provider.proxy)

    users = []
    for user in USERS:
        user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
        user_account.address = WrapperAddress(user)
        user_account.sync_nonce(context.network_provider.proxy)
        users.append(user_account)

    return users

Chaim Simulator Stack

In [ ]:
import subprocess
from time import sleep

CS_DOCKER_PATH = Path.home() / "projects/testing/full-stack-docker-compose/chain-simulator"

def start_chain_sim_stack():
    # stop first in case one is already running
    print("Initializing chain simulator...")
    p = subprocess.Popen(["docker", "compose", "down"], cwd = CS_DOCKER_PATH, stdout=subprocess.DEVNULL, 
        stderr=subprocess.PIPE)
    p.wait()
    
    p = subprocess.Popen(["docker", "compose", "up", "-d"], cwd = CS_DOCKER_PATH, stdout=subprocess.DEVNULL, 
        stderr=subprocess.PIPE)
    sleep(60)
    return p

def stop_chain_sim_stack(p):
    print("Stopping chain simulator...")
    p.terminate()
    p = subprocess.Popen(["docker", "compose", "down"], cwd = CS_DOCKER_PATH, stdout=subprocess.DEVNULL, 
        stderr=subprocess.PIPE)
    p.wait()
    _ = subprocess.run(["docker", "system", "prune", "-f"], cwd = CS_DOCKER_PATH, stdout=subprocess.DEVNULL, 
        stderr=subprocess.PIPE)
    print("Chain simulator stopped.")

Upgrade farm

In [ ]:
def farm_upgrade():
    tx_hash = farm_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                         "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.3.1-rc1/farm-staking.wasm", [], True)

    advance_blocks(1)
    tx_hash = farm_contract.resume(context.deployer_account, context.network_provider.proxy)
    advance_blocks(1)

    code_hash = context.network_provider.proxy.get_account(WrapperAddress(farm_contract.address)).code_hash
    assert base64_to_hex(code_hash) == "551a8a2ac4265a60109e270008eeb4f1804dc5e4ca3080819b0108f3de62322b"

Stake farm consolidated

In [ ]:
from utils.utils_chain import Account, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

def stake_farm_for_user(user_account: Account):
    user_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farming_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farm_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farm_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    event = EnterFarmEvent(farm_contract.farming_token, 0, farming_tk_balance,
                       farm_contract.farm_token, farm_tk_nonce, farm_tk_balance)

    tx_hash = farm_contract.stake_farm(context.network_provider, user_account, event)
    return tx_hash

Stake farm no consolidation

In [ ]:
from utils.utils_chain import Account, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

def stake_farm_no_consolidation_for_user(user_account: Account):
    user_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farming_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    event = EnterFarmEvent(farm_contract.farming_token, 0, farming_tk_balance,
                       "", 0, 0)

    tx_hash = farm_contract.stake_farm(context.network_provider, user_account, event, True)
    return tx_hash

Unstake

In [ ]:
from events.farm_events import ExitFarmEvent


def unstake_for_user(user_account:Account):
    user_account.sync_nonce(context.network_provider.proxy)

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farm_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farm_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")
    
    event = ExitFarmEvent(farm_contract.farm_token, farm_tk_balance, farm_tk_nonce, [], None)

    tx_hash = farm_contract.unstake_farm(context.network_provider, user_account, event)
    return tx_hash

Unbond

In [ ]:
from events.farm_events import ExitFarmEvent


def unbond_for_user(user_account:Account):
    user_account.sync_nonce(context.network_provider.proxy)

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farm_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farm_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")
    
    event = ExitFarmEvent(farm_contract.farm_token, farm_tk_balance, farm_tk_nonce, [], None)

    tx_hash = farm_contract.unbond_farm(context.network_provider, user_account, event)
    return tx_hash

Claim rewards

In [ ]:
from contracts.farm_contract import ClaimRewardsFarmEvent

def claim_rewards_user(user_account):
    user_account.sync_nonce(context.network_provider.proxy)

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farm_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farm_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    event = ClaimRewardsFarmEvent(farm_tk_balance, farm_tk_nonce, '')

    tx_hash = farm_contract.claim_rewards(context.network_provider, user_account, event)
    return tx_hash

Claim boosted rewards

In [ ]:
def claim_boosted_rewards_user(user_account):
    user_account.sync_nonce(context.network_provider.proxy)

    event = ClaimRewardsFarmEvent(0, 0, '', user_account.address.bech32())

    tx_hash = farm_contract.claim_boosted_rewards(context.network_provider, user_account, event)
    return tx_hash

Compound rewards

In [ ]:
from events.farm_events import CompoundRewardsFarmEvent


def compound_rewards_user(user_account):
    user_account.sync_nonce(context.network_provider.proxy)

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farm_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farm_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    event = CompoundRewardsFarmEvent(farm_tk_balance, farm_tk_nonce)

    tx_hash = farm_contract.compound_rewards(context.network_provider, user_account, event)
    return tx_hash

Stake farm on behalf no consolidation

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

def enter_farm_on_behalf_no_consolidation_for_user(caller_account: Account, user_account: Account):
    caller_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farming_token, caller_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    event = EnterFarmEvent(farm_contract.farming_token, 0, farming_tk_balance,
                           "", 0, 0, user_account.address.bech32())

    tx_hash = farm_contract.stake_farm_on_behalf(context.network_provider, caller_account, event)
    return tx_hash

Stake farm on behalf with previous position

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

def enter_farm_on_behalf_for_user(caller_account: Account, user_account: Account, position_balance: int):
    caller_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farming_token, caller_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farm_token, caller_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farm_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) == position_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    event = EnterFarmEvent(farm_contract.farming_token, 0, farming_tk_balance,
                       farm_contract.farm_token, farm_tk_nonce, farm_tk_balance, user_account.address.bech32())

    tx_hash = farm_contract.stake_farm_on_behalf(context.network_provider, caller_account, event)
    return tx_hash

Claim on behalf

In [ ]:
from utils.utils_chain import Account, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

def claim_on_behalf_from_user(claim_account: Account, user_account: Account, position_balance: int):
    claim_account.sync_nonce(context.network_provider.proxy)

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farm_token, claim_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farm_token} in account')
    for token in tokens_in_account:
        if int(token['balance']) == position_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    event = ClaimRewardsFarmEvent(farm_tk_balance, farm_tk_nonce, '', user_account.address.bech32())

    tx_hash = farm_contract.claim_rewards_on_behalf(context.network_provider, claim_account, event)
    return tx_hash

Get user positions

In [ ]:
from utils.utils_tx import multi_esdt_transfer, ESDTToken
from utils.utils_chain import Account

def get_single_user_position(user_account):
    user_account.sync_nonce(context.network_provider.proxy)

    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farm_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farm tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > 0:
            farm_tk = token
            break
    token = ESDTToken(farm_tk['tokenIdentifier'], farm_tk['nonce'], int(farm_tk['balance']))
    return token

In [ ]:
from typing import Dict


def get_user_positions(user_account: Account) -> List[Dict[str, Any]]:
    positions = []
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farm_token, user_account.address.bech32(), context.network_provider.proxy)
    for token in tokens_in_account:
        if int(token['balance']) > 0:
            positions.append(token)
    return positions

Transfer position

In [ ]:
from utils.utils_tx import multi_esdt_transfer, ESDTToken

def transfer_position(user_account, destination):
    user_account.sync_nonce(context.network_provider.proxy)

    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farm_token, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farm tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > 0:
            farm_tk = token
            break
    token = ESDTToken(farm_tk['tokenIdentifier'], farm_tk['nonce'], int(farm_tk['balance']))
    print(f'Sending: {token.get_token_data()} from {user_account.address.bech32()}')
    multi_esdt_transfer(context.network_provider.proxy, 1000000, user_account, destination.address, [token])

Utils

In [ ]:
from typing import Dict, Any, List, Tuple, Optional
from dataclasses import dataclass
from enum import Enum, auto
from collections import defaultdict

class DictType(Enum):
    USER_FARM_STATS_1 = auto()
    USER_FARM_STATS_1_BEHALF = auto()
    FARM_CONTRACT_STATS_1 = auto()
    USER_POSITION_INITIAL = auto()
    USER_POSITION_FINAL = auto()
    USER_FARM_STATS_2 = auto()
    USER_FARM_STATS_2_BEHALF = auto()
    FARM_CONTRACT_STATS_2 = auto()
    OP1 = auto()
    OP2 = auto()
    OP3 = auto()
    OP4 = auto()

@dataclass
class DictComparison:
    before: Dict[str, Any]
    after: Dict[str, Any]
    description: str
    phase: str

class DictCollector:
    def __init__(self):
        self.reset()

    def reset(self):
        """Reset all collections for a new scenario"""
        self.before_upgrade: Dict[DictType, List[Tuple[Dict[str, Any], str]]] = defaultdict(list)
        self.after_upgrade: Dict[DictType, List[Tuple[Dict[str, Any], str]]] = defaultdict(list)
        self.current_phase = "before"  # "before" or "after"
    
    def set_phase(self, phase: str):
        """Set the current collection phase"""
        if phase not in ["before", "after"]:
            raise ValueError("Phase must be 'before' or 'after'")
        self.current_phase = phase

    def add(self, dict_type: DictType, dict_data: Dict[str, Any], description: str = ""):
        """Add a dictionary to the current phase collection"""
        if self.current_phase == "before":
            self.before_upgrade[dict_type].append((dict_data, description))
        else:
            self.after_upgrade[dict_type].append((dict_data, description))

    def _compare_dicts(self, dict1: Any, dict2: Any) -> Tuple[bool, Optional[str]]:
        """
        Compare two objects that can be either dictionaries or lists of dictionaries.
        Returns (is_equal, difference_description)
        """
        # Handle lists of dictionaries
        if isinstance(dict1, list) and isinstance(dict2, list):
            if len(dict1) != len(dict2):
                return False, f"Different list lengths: {len(dict1)} != {len(dict2)}"
            
            for i, (item1, item2) in enumerate(zip(dict1, dict2)):
                if isinstance(item1, (dict, list)) and isinstance(item2, (dict, list)):
                    is_equal, diff = self._compare_dicts(item1, item2)
                    if not is_equal:
                        return False, f"List item {i} difference: {diff}"
                elif item1 != item2:
                    return False, f"List item {i} mismatch: {item1} != {item2}"
            return True, None

        # Handle dictionaries
        if isinstance(dict1, dict) and isinstance(dict2, dict):
            if dict1.keys() != dict2.keys():
                missing_keys = set(dict1.keys()) - set(dict2.keys())
                extra_keys = set(dict2.keys()) - set(dict1.keys())
                return False, f"Different keys. Missing: {missing_keys}, Extra: {extra_keys}"

            for key in dict1:
                if isinstance(dict1[key], (dict, list)) and isinstance(dict2[key], (dict, list)):
                    is_equal, diff = self._compare_dicts(dict1[key], dict2[key])
                    if not is_equal:
                        return False, f"Nested difference at key '{key}': {diff}"
                elif dict1[key] != dict2[key]:
                    return False, f"Value mismatch for key '{key}': {dict1[key]} != {dict2[key]}"
            return True, None

        # Handle case where types don't match
        if type(dict1) != type(dict2):
            return False, f"Type mismatch: {type(dict1)} != {type(dict2)}"

        # Handle other cases
        return dict1 == dict2, f"Value mismatch: {dict1} != {dict2}" if dict1 != dict2 else None

    def compare_all(self) -> List[str]:
        """
        Compare all collected dictionary pairs and return a list of differences found.
        """
        differences = []

        for dict_type in DictType:
            before_list = self.before_upgrade[dict_type]
            after_list = self.after_upgrade[dict_type]

            # Check if we have matching pairs
            if len(before_list) != len(after_list):
                differences.append(
                    f"{dict_type.name}: Mismatched number of collections - "
                    f"Before: {len(before_list)}, After: {len(after_list)}"
                )
                continue

            # Compare each pair
            for i, ((before_dict, before_desc), (after_dict, after_desc)) in enumerate(zip(before_list, after_list)):
                is_equal, diff = self._compare_dicts(before_dict, after_dict)
                if not is_equal:
                    diff_msg = f"{dict_type.name} comparison failed"
                    if before_desc or after_desc:
                        diff_msg += f" (Before: {before_desc}, After: {after_desc})"
                    diff_msg += f": {diff}"
                    differences.append(diff_msg)

        return differences
    
    def print_collections(self):
        """
        Print a formatted view of all collections before and after upgrade.
        """
        print("\nCollections Summary:")
        print("=" * 80)

        for dict_type in DictType:            
            before_list = self.before_upgrade[dict_type]
            after_list = self.after_upgrade[dict_type]

            if not before_list and not after_list:
                continue

            print(f"\n{dict_type.name}:")
            print("-" * 40)

            print("\nBefore upgrade:")
            if not before_list:
                print("  No collections")
            for i, (data, desc) in enumerate(before_list, 1):
                print(f"  Collection {i}" + (f" ({desc})" if desc else ""))
                if isinstance(data, dict):
                    for key, value in data.items():
                        print(f"    {key}: {value}")
                else:
                    print(f"    {data}")

            print("\nAfter upgrade:")
            if not after_list:
                print("  No collections")
            for i, (data, desc) in enumerate(after_list, 1):
                print(f"  Collection {i}" + (f" ({desc})" if desc else ""))
                if isinstance(data, dict):
                    for key, value in data.items():
                        print(f"    {key}: {value}")
                else:
                    print(f"    {data}")

            print("\n" + "=" * 80)

# Scenarios

Enter farm no previous position

In [ ]:
user_index = 1
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = stake_farm_no_consolidation_for_user(user)
advance_blocks(5)
sleep(2)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial stake farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

collector.set_phase("after")


u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = stake_farm_no_consolidation_for_user(user)
advance_blocks(5)
sleep(2)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial stake operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Enter farm with previous position

In [ ]:
user_index = 0
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = stake_farm_for_user(user)
advance_blocks(5)
sleep(2)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial stake farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

collector.set_phase("after")


u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = stake_farm_for_user(user)
advance_blocks(5)
sleep(2)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial stake farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Unstake

In [ ]:
user_index = 0
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)
up0 = get_user_positions(user)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")
collector.add(DictType.USER_POSITION_INITIAL, up0, "Initial user positions")

tx_hash = unstake_for_user(user)
advance_blocks(5)
sleep(2)
unstake_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, unstake_op_1, "Initial unstake operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)
up1 = get_user_positions(user)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")
collector.add(DictType.USER_POSITION_FINAL, up1, "Ending user positions")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

collector.set_phase("after")


u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)
up0 = get_user_positions(user)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")
collector.add(DictType.USER_POSITION_INITIAL, up0, "Initial user positions")

tx_hash = unstake_for_user(user)
advance_blocks(5)
sleep(2)
unstake_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, unstake_op_1, "Initial unstake operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)
up1 = get_user_positions(user)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")
collector.add(DictType.USER_POSITION_FINAL, up1, "Ending user positions")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Unbond

In [ ]:
user_index = 0
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)
up0 = get_user_positions(user)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")
collector.add(DictType.USER_POSITION_INITIAL, up0, "Initial user positions")

tx_hash = unstake_for_user(user)
advance_blocks(5)
sleep(2)
unstake_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, unstake_op_1, "Initial unstake operation")

tx_hash = unbond_for_user(user)
advance_blocks(5)
sleep(2)
unbond_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP2, unbond_op_1, "Initial unbond operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)
up1 = get_user_positions(user)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")
collector.add(DictType.USER_POSITION_FINAL, up1, "Ending user positions")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

collector.set_phase("after")


u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)
up0 = get_user_positions(user)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")
collector.add(DictType.USER_POSITION_INITIAL, up0, "Initial user positions")

tx_hash = unstake_for_user(user)
advance_blocks(5)
sleep(2)
unstake_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, unstake_op_1, "Initial unstake operation")

tx_hash = unbond_for_user(user)
advance_blocks(5)
sleep(2)
unbond_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP2, unbond_op_1, "Initial unbond operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)
up1 = get_user_positions(user)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")
collector.add(DictType.USER_POSITION_FINAL, up1, "Ending user positions")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Claim rewards

In [ ]:
user_index = 0
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = claim_rewards_user(user)
advance_blocks(5)
sleep(2)
claim_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_op_1, "Initial claim farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

collector.set_phase("after")


u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = claim_rewards_user(user)
advance_blocks(5)
sleep(2)
claim_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_op_1, "Initial claim operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Claim rewards multiple positions

In [ ]:
user_index = 2
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_for_user(user)
advance_blocks(5)
sleep(2)

enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm operation")
up1 = get_user_positions(user)
collector.add(DictType.USER_POSITION_INITIAL, up1, "Initial user positions")

tx_hash = claim_rewards_user(user)
advance_blocks(5)
sleep(2)
claim_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP2, claim_op_1, "Initial claim farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)
up2 = get_user_positions(user)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")
collector.add(DictType.USER_POSITION_FINAL, up2, "Ending user positions")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

collector.set_phase("after")


u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_for_user(user)
advance_blocks(5)
sleep(2)

enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm operation")
up1 = get_user_positions(user)
collector.add(DictType.USER_POSITION_INITIAL, up1, "Initial user positions")

tx_hash = claim_rewards_user(user)
advance_blocks(5)
sleep(2)
claim_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP2, claim_op_1, "Initial claim operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)
up2 = get_user_positions(user)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")
collector.add(DictType.USER_POSITION_FINAL, up2, "Ending user positions")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Claim boosted rewards

In [ ]:
user_index = 0
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = claim_boosted_rewards_user(user)
advance_blocks(5)
sleep(2)
claim_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_op_1, "Initial claim boosted farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

collector.set_phase("after")


u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = claim_boosted_rewards_user(user)
advance_blocks(5)
sleep(2)
claim_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, claim_op_1, "Initial claim boosted operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Compound rewards

In [ ]:
user_index = 0
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)
up1 = get_user_positions(user)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")
collector.add(DictType.USER_POSITION_INITIAL, up1, "Initial user positions")

tx_hash = compound_rewards_user(user)
advance_blocks(5)
sleep(2)
compound_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, compound_op_1, "Initial compound farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)
up2 = get_user_positions(user)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")
collector.add(DictType.USER_POSITION_FINAL, up2, "Ending user positions")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

collector.set_phase("after")


u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)
up1 = get_user_positions(user)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")
collector.add(DictType.USER_POSITION_INITIAL, up1, "Initial user positions")

tx_hash = compound_rewards_user(user)
advance_blocks(5)
sleep(2)
compound_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, compound_op_1, "Initial compound farm operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)
up2 = get_user_positions(user)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")
collector.add(DictType.USER_POSITION_FINAL, up2, "Ending user positions")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Stake farm on behalf no previous position

In [ ]:
user_index = 0
whitelisted_user_index = 1
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

whitelisted_user = users_init()[whitelisted_user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [whitelisted_user.address.bech32()])
advance_blocks(5)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_on_behalf_no_consolidation_for_user(whitelisted_user, user)
advance_blocks(5)
sleep(6)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm on behalf operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
whitelisted_user = users_init()[whitelisted_user_index]

advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [whitelisted_user.address.bech32()])
advance_blocks(5)

collector.set_phase("after")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_on_behalf_no_consolidation_for_user(whitelisted_user, user)
advance_blocks(5)
sleep(6)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm on behalf operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Stake farm on behalf with previous position

In [ ]:
user_index = 0
whitelisted_user_index = 1
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

whitelisted_user = users_init()[whitelisted_user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

token = get_single_user_position(user)
advance_blocks(5)

transfer_position(user, whitelisted_user)
advance_blocks(5)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [whitelisted_user.address.bech32()])
advance_blocks(5)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_on_behalf_for_user(whitelisted_user, user, token.token_amount)
advance_blocks(5)
sleep(6)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm on behalf operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
whitelisted_user = users_init()[whitelisted_user_index]

advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

token = get_single_user_position(user)
advance_blocks(5)

transfer_position(user, whitelisted_user)
advance_blocks(5)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [whitelisted_user.address.bech32()])
advance_blocks(5)

collector.set_phase("after")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_on_behalf_for_user(whitelisted_user, user, token.token_amount)
advance_blocks(5)
sleep(6)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm on behalf operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Stake farm on behalf blacklisted

In [ ]:
user_index = 0
blacklisted_user_index = 1
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

blacklisted_user = users_init()[blacklisted_user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

token = get_single_user_position(user)
advance_blocks(5)

transfer_position(user, blacklisted_user, token.token_amount)
advance_blocks(5)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_on_behalf_no_consolidation_for_user(blacklisted_user, user)
advance_blocks(5)
sleep(6)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm on behalf operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
blacklisted_user = users_init()[blacklisted_user_index]

advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

collector.set_phase("after")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = enter_farm_on_behalf_no_consolidation_for_user(blacklisted_user, user)
advance_blocks(5)
sleep(6)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial enter farm on behalf operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Claim rewards on behalf

In [ ]:
user_index = 0
whitelisted_user_index = 1
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

whitelisted_user = users_init()[whitelisted_user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

token = get_single_user_position(user)
advance_blocks(5)

transfer_position(user, whitelisted_user)
advance_blocks(5)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [whitelisted_user.address.bech32()])
advance_blocks(5)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = claim_on_behalf_from_user(whitelisted_user, user, token.token_amount)
advance_blocks(5)
sleep(6)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial claim on behalf operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
whitelisted_user = users_init()[whitelisted_user_index]

advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

token = get_single_user_position(user)
advance_blocks(5)

transfer_position(user, whitelisted_user)
advance_blocks(5)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

permissions_hub_contract.add_to_whitelist(user, context.network_provider.proxy, [whitelisted_user.address.bech32()])
advance_blocks(5)

collector.set_phase("after")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = claim_on_behalf_from_user(whitelisted_user, user, token.token_amount)
advance_blocks(5)
sleep(6)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial claim on behalf operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)

Claim rewards on behalf blacklisted

In [ ]:
user_index = 0
blacklisted_user_index = 1
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]

blacklisted_user = users_init()[blacklisted_user_index]

initial_blocks = 10
advance_blocks(initial_blocks)

token = get_single_user_position(user)
advance_blocks(5)

transfer_position(user, blacklisted_user)
advance_blocks(5)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

pprint.pprint(farm_contract.get_config_dict())

collector = DictCollector()
collector.set_phase("before")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = claim_on_behalf_from_user(blacklisted_user, user, token.token_amount)
advance_blocks(5)
sleep(6)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial claim on behalf operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(whitelisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

# ------------------------------------------------------------------------------------------------
stop_chain_sim_stack(chain_sim)
chain_sim = start_chain_sim_stack()
setup_chain_sim()
user = users_init()[user_index]
blacklisted_user = users_init()[blacklisted_user_index]

advance_blocks(1)
consumed_blocks = 1

farm_upgrade()  # eats 2 blocks
consumed_blocks += 2

block_diff = initial_blocks - consumed_blocks
advance_blocks(block_diff)

token = get_single_user_position(user)
advance_blocks(5)

transfer_position(user, blacklisted_user)
advance_blocks(5)

permissions_hub_contract = deploy_permissions_hub()
advance_blocks(5)

set_permissions_hub(permissions_hub_contract)
advance_blocks(5)

collector.set_phase("after")

u11 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u12 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c11 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_1, u11, "Initial farm user stats")
collector.add(DictType.USER_FARM_STATS_1_BEHALF, u12, "Initial farm user stats on behalf")
collector.add(DictType.FARM_CONTRACT_STATS_1, c11, "Initial farm contract stats")

tx_hash = claim_on_behalf_from_user(blacklisted_user, user, token.token_amount)
advance_blocks(5)
sleep(6)
enter_op_1 = context.network_provider.get_tx_operations(tx_hash, True)
collector.add(DictType.OP1, enter_op_1, "Initial claim on behalf operation")

u21 = farm_contract.get_all_user_boosted_stats(user.address.bech32(), context.network_provider.proxy)
u22 = farm_contract.get_all_user_boosted_stats(blacklisted_user.address.bech32(), context.network_provider.proxy)
c21 = farm_contract.get_all_stats(context.network_provider.proxy)

collector.add(DictType.USER_FARM_STATS_2, u21, "Ending farm user stats")
collector.add(DictType.USER_FARM_STATS_2_BEHALF, u22, "Ending farm user on behalf stats")
collector.add(DictType.FARM_CONTRACT_STATS_2, c21, "Ending farm contract stats")

differences = collector.compare_all()
if differences:
    print("Found differences:")
    for diff in differences:
        print(f"- {diff}")
else:
    print("All comparisons passed!")
collector.print_collections()

stop_chain_sim_stack(chain_sim)